In [ ]:
from pathlib import Path
from datetime import datetime
import time, traceback
import nbformat
from nbclient import NotebookClient

# ========= 設定 =========
# 1) 明示順で実行
# 05_get_data_from.ipynbは長時間稼働のため、個別実行を推奨
RUN_LIST = [
    "05_get_data_from.ipynb",
    "10_col_rename.ipynb",
    "15_drop_duplicate.ipynb",
    "17_check_digit.ipynb",
    "20_calc_cost_duration.ipynb",
    "25_assign_region.ipynb",
    "30_assign_cat_dummy.ipynb",
    "35_assign_wgi.ipynb",
    "40_assign_freedomrate.ipynb",
    "45_assign_gdp_at_pjt_start.ipynb",
    "45_assign_gdp.ipynb",
    "50_assign_population.ipynb",
    "55_assign_ex_eval_flg.ipynb",
    "60_assign_toughness.ipynb",
    "60_mod_assign_toughness.ipynb",
    "65_unify_sectors.ipynb",
    "99_rename_df.ipynb",
]
# 2) 自動検出にしたい場合は RUN_LIST = [] とし、
#    00_main.ipynb を除く「NN_*.ipynb」を数値昇順で実行
AUTO_GLOB = "[0-9][0-9]_*.ipynb"
STOP_ON_ERROR = True            # エラーで停止（Falseなら続行）
KERNEL_NAME = None              # None ならノートブックのkernelspecを使用
TIMEOUT_SEC = 60 * 20           # 各ノートブックの実行タイムアウト
OUTPUT_DIR = Path("_executed")  # 実行後のノートブック保存先
OUTPUT_DIR.mkdir(exist_ok=True)

# ========= 実行対象の決定 =========
root = Path(".")
if not RUN_LIST:
    # 自動検出: 00_main.ipynb を除外し、数値プレフィックスでソート
    cands = [p.name for p in sorted(root.glob(AUTO_GLOB))]
    RUN_LIST = [n for n in cands if n != "00_main.ipynb"]

print("Run order:")
for n in RUN_LIST:
    print(" -", n)

# ========= 実行関数 =========
def run_notebook(nb_path: Path) -> dict:
    nb = nbformat.read(nb_path.open("r", encoding="utf-8"), as_version=4)
    kernel = KERNEL_NAME or nb.metadata.get("kernelspec", {}).get("name", "python3")
    client = NotebookClient(
        nb,
        timeout=TIMEOUT_SEC,
        kernel_name=kernel,
        allow_errors=False,
        record_timing=True,
    )
    t0 = time.time()
    client.execute()
    dt = time.time() - t0

    stamp = datetime.now().strftime("%Y%m%d-%H%M%S")
    out_path = OUTPUT_DIR / f"{nb_path.stem}__run-{stamp}.ipynb"
    with out_path.open("w", encoding="utf-8") as f:
        nbformat.write(nb, f)

    return {"name": nb_path.name, "status": "ok", "secs": round(dt, 1), "output": str(out_path)}

# ========= メインループ =========
results = []
for name in RUN_LIST:
    path = root / name
    print(f"\n=== Running: {name} ===")
    try:
        res = run_notebook(path)
        print(f"✅ OK  ({res['secs']}s) → {res['output']}")
        results.append(res)
    except Exception as e:
        tb = traceback.format_exc(limit=3)
        print(f"❌ FAIL: {name}\n{tb}")
        results.append({"name": name, "status": "fail", "secs": None, "output": "", "error": str(e)})
        if STOP_ON_ERROR:
            break

# ========= サマリ表示 =========
try:
    import pandas as pd
    df_res = pd.DataFrame(results)
    display(df_res)
except Exception:
    print(results)